In [ ]:
#using Revise

In [ ]:
using Knet

In [ ]:
using ImageView;

In [ ]:
include("../helper.jl")
include("mpii.annotation.parser.jl")
include("accuracy.jl")
include("loss.jl")
include("mpii.batcher.jl")
include("../models.jl")
include("../training.jl")
include("../plots.jl")
include("../modular.resnet.jl")

In [ ]:
dtrn, dval = get_mpii_batches(32)

In [ ]:
# 5) Implementing a Convolutional Neural Network
simple_conv_net =   Chain(
    ResLayerConv(3, 3, 3, 8; stride=2, padding=1),
    ResLayerConv(3, 3, 8, 16; stride=2, padding=1),
    ResLayerConv(3, 3, 16, 32; stride=2, padding=1),
    Conv2(3, 3, 32, 32),
    ResLayerConv(3, 3, 32, 32; stride=2, padding=1),
    Deconv(3, 3, 14, 32; padding=1),
    loss=sigmoid_cross_entropy_loss
)

In [ ]:
simple_conv_net_results = trainresults("simple_conv_net.jld2", simple_conv_net, dtrn, dval, 100, 0.15; accuracy_func=modelized_naive_pck_sigm, error_func=nothing);

In [ ]:
draw_plots(simple_conv_net_results)

In [ ]:
dtrn_x1, dtrn_y1 = first(dtrn)
dval_x1, dval_y1 = first(dval)

In [ ]:
simple_conv_net(dval)

In [ ]:
sigmoided_output = Array(sigm.(simple_conv_net(dval_x1)));

In [ ]:
total_pck = naive_pck(sigmoided_output, dval_y1);
println(total_pck)

In [ ]:
function get_params(params, atype)
    len = length(params["value"])
    ws, ms = [], []
    for k = 1:len
        name = params["name"][k]
        value = convert(Array{Float32}, params["value"][k])

        if endswith(name, "moments")
            push!(ms, reshape(value[:,1], (1,1,size(value,1),1)))
            push!(ms, reshape(value[:,2], (1,1,size(value,1),1)))
        elseif startswith(name, "bn")
            push!(ws, reshape(value, (1,1,length(value),1)))
        elseif startswith(name, "fc") && endswith(name, "filter")
            push!(ws, transpose(reshape(value,(size(value,3),size(value,4)))))
        elseif startswith(name, "conv") && endswith(name, "bias")
            push!(ws, reshape(value, (1,1,length(value),1)))
        else
            push!(ws, value)
        end
    end
    map(wi->convert(atype, wi), ws),
    map(mi->convert(atype, mi), ms)
end

In [ ]:
model_file_path = "/home/gsoykan/Desktop/comp541/comp541_term_project/results/imagenet-resnet-152-dag.mat"

o = Dict(
  :atype => KnetArray{Float32},
  :model => model_file_path,
  :top   => 10
)
model = matread(abspath(o[:model]))
w, ms = get_params(model["params"], o[:atype]);

In [ ]:
modular_resnet152 = generate_headless_resnet_from_weights(w, ms);

In [ ]:
resnet_with_deconv = Chain(modular_resnet152.layers...,  
    Deconv(3, 3, 14, 2048; padding=1); loss=sigmoid_cross_entropy_loss);

In [ ]:
resnet_output = Array(sigm.(resnet_with_deconv(dtrn_x1)))

In [ ]:
total_pck_resnet = naive_pck(resnet_output, dtrn_y1);
println(total_pck_resnet)

In [ ]:
resnet_with_deconv(dtrn)

In [ ]:
CUDA.reclaim()

In [ ]:
CUDA.memory_status() 

In [ ]:
GC.gc(true)

In [ ]:
# TODO: Fix batch norm issue here try with different batchnorms
# Check here https://github.com/denizyuret/Knet.jl/blob/master/examples/dcgan-mnist/dcgan.jl
# and
# check previous batch norm emails

resnetnet_results = trainresults("resnet_with_deconv.jld2", resnet_with_deconv, dtrn, dval, 100, 0.15; accuracy_func=nothing, error_func=nothing);

In [ ]:
sigmoided_output = Array(sigm.(simple_conv_net(dtrn_x1)));

In [ ]:
function draw_first_images_forehead(y)
test_y = y[:, :, :, 17];
colorview(Gray, test_y[:, : , 2])
end

In [ ]:
#Array(dtrn_y1)

In [ ]:
draw_first_images_forehead(Array(dtrn_y1))

In [ ]:
draw_first_images_forehead(sigmoided_output)

In [ ]:
#draw_plots(simple_conv_net_results);

In [ ]:
test_image = Array(dtrn_x1)[:, :, :, 17];

In [ ]:
perm = permutedims(test_image, [3, 1, 2])
colorview(RGB, perm)
#colorview(RGB, rand(3,256,256))


In [ ]:
size(ytrn)

In [ ]:
first(dtrn)[1] |> size

In [ ]:
first(dtrn)[2] |> size

In [ ]:
#julia> w0 = KnetArray(randn(Float32, 5, 5, 1, 20));

#julia> x0 = KnetArray(rand(Float32, 24, 24, 20, 100));

#julia> size(deconv4(w0,x0)) # this output does not make it sense, it should give what I want
#(28, 28, 20, 100)
#

w0 = KnetArray(randn(Float32, 5, 5, 1, 20));
 x0 = KnetArray(rand(Float32, 24, 24, 20, 100));

 # this output does not make it sense, it should give what I want
#(28, 28, 20, 100)

In [ ]:
size(deconv4(w0,x0))

In [ ]:
tc = Conv2(3, 3, 3, 3)

In [ ]:
fb = first(dtrn)[1];

In [ ]:
res_tc = tc2(fb) |> tc;

In [ ]:
mid = res_tc |> tc2

In [ ]:
w1 = KnetArray(randn(Float32, 3,3, 1, 3));

In [ ]:
#deconv4(w1, mid)
size(deconv4(w1, mid; padding=1))

In [ ]:
tc2 = ResLayerConv(3, 3, 3, 3; stride=2, padding=1)

In [ ]:
res_tc2 = res_tc |> tc2;

In [ ]:
size(res_tc)

In [ ]:
tc2(fb) |> tc2 |> tc2;